In [1]:
!pip install contractions
!pip install googletrans==4.0.0-rc1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=8b847bfc130fa7b8f400ee9fef54c5164759d595d9b4a136b886fc4ff856db13
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498

##Lematización:

- La lematización es el proceso de reducir una palabra a su forma base o raíz, conocida como lema. Por ejemplo, las palabras "corriendo", "corre" y "correrá" tienen el mismo lema "correr".
- El objetivo de la lematización es normalizar las palabras para que las variantes de una misma palabra se consideren iguales en el análisis, lo que ayuda a mejorar la precisión de los modelos de NLP.

##Stopwords:

- Las stopwords son palabras muy comunes en un idioma que a menudo no aportan un significado adicional al texto y se pueden eliminar sin afectar la comprensión del mismo. Ejemplos de stopwords en inglés son "the", "and", "is", etc.
- La eliminación de stopwords es importante para reducir el ruido en los datos y mejorar la eficiencia de los modelos de NLP al centrarse en las palabras más significativas del texto.

In [2]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import re
import string
import contractions
import multiprocessing
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

##Limpieza del texto

- Normalización de datos

- Eliminación de ruido

- Reducción del vocabulario

- Mejora de la precisión

*clean_text_round1*: Limpia el texto convirtiéndolo a minúsculas, eliminando texto entre corchetes, signos de puntuación y palabras con números.

*clean_text_round2*: Elimina comillas, puntos suspensivos, y saltos de línea del texto.

In [3]:
def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def clean_text_round2(text):
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text

In [4]:
lemmatizer = WordNetLemmatizer()
sw = stopwords.words('english')

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial/Materia/2024/simpsons_dataset.csv')
print(df.head())

# Asegurarse de que todas las entradas en 'spoken_words' son cadenas de texto
df['spoken_words'] = df['spoken_words'].astype(str)

# Aplicar las funciones de limpieza usando funciones lambda (rounds)
round0 = lambda x: contractions.fix(x)
round1 = lambda x: clean_text_round1(x)
round2 = lambda x: clean_text_round2(x)

# Aplicar cada ronda de limpieza secuencialmente
df['spoken_words'] = df['spoken_words'].apply(round0)
df['spoken_words'] = df['spoken_words'].apply(round1)
df['spoken_words'] = df['spoken_words'].apply(round2)


df.head()

        raw_character_text                                       spoken_words
0              Miss Hoover  No, actually, it was a little of both. Sometim...
1             Lisa Simpson                             Where's Mr. Bergstrom?
2              Miss Hoover  I don't know. Although I'd sure like to talk t...
3             Lisa Simpson                         That life is worth living.
4  Edna Krabappel-Flanders  The polls will be open from now until the end ...


,raw_character_text,spoken_words
0,Miss Hoover,no actually it was a little of both sometim...
1,Lisa Simpson,where is mr bergstrom
2,Miss Hoover,i do not know although i would sure like to t...
3,Lisa Simpson,that life is worth living
4,Edna Krabappel-Flanders,the polls will be open from now until the end ...


###Tokenización de las frases

- Convierte cada frase en una lista de palabras, tokenizando el texto por espacios. input_data se convierte en una lista de listas de palabras.

In [6]:
input_data = [row.split() for row in df['spoken_words']]
input_data[0]

['no',
 'actually',
 'it',
 'was',
 'a',
 'little',
 'of',
 'both',
 'sometimes',
 'when',
 'a',
 'disease',
 'is',
 'in',
 'all',
 'the',
 'magazines',
 'and',
 'all',
 'the',
 'news',
 'shows',
 'it',
 'is',
 'only',
 'natural',
 'that',
 'you',
 'think',
 'you',
 'have',
 'it']

###Construcción de bigramas

**"El gato está sobre la mesa"**

Los bigramas de esta oración serían:

- "El gato"
- "gato está"
- "está sobre"
- "sobre la"
- "la mesa"

Cada bigrama representa dos palabras consecutivas en la oración original.
  
Captura de Contexto Local   #New York




In [7]:
phrases = Phrases(input_data, min_count=30, progress_per=1000)
bigram = Phraser(phrases)
sentences = bigram[input_data]
sentences[0]

['no',
 'actually',
 'it',
 'was',
 'a',
 'little',
 'of',
 'both',
 'sometimes',
 'when',
 'a',
 'disease',
 'is',
 'in',
 'all',
 'the',
 'magazines',
 'and',
 'all',
 'the',
 'news',
 'shows',
 'it',
 'is',
 'only',
 'natural',
 'that',
 'you',
 'think',
 'you',
 'have',
 'it']

In [8]:
for sentence in sentences:
    for word in sentence:
        if "_" in word:
            print(f"Bigrama encontrado: {word}")

Streaming output truncated to the last 5000 lines.
Bigrama encontrado: cannot_believe
Bigrama encontrado: next_week
Bigrama encontrado: an_hour
Bigrama encontrado: have_been
Bigrama encontrado: kind_of
Bigrama encontrado: so_fast
Bigrama encontrado: do_not
Bigrama encontrado: excuse_me
Bigrama encontrado: have_been
Bigrama encontrado: do_not
Bigrama encontrado: do_not
Bigrama encontrado: let_us
Bigrama encontrado: your_father
Bigrama encontrado: do_not
Bigrama encontrado: have_been
Bigrama encontrado: am_glad
Bigrama encontrado: even_though
Bigrama encontrado: talking_about
Bigrama encontrado: talking_about
Bigrama encontrado: do_not
Bigrama encontrado: how_bout
Bigrama encontrado: do_not
Bigrama encontrado: each_other
Bigrama encontrado: do_not
Bigrama encontrado: am_sorry
Bigrama encontrado: do_not
Bigrama encontrado: woo_hoo
Bigrama encontrado: da_da
Bigrama encontrado: da_da
Bigrama encontrado: da_da
Bigrama encontrado: take_care
Bigrama encontrado: worse_than
Bigrama encontrado: a

###Construcción del vocabulario

In [9]:
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=20, window=2, vector_size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=cores)

In [11]:
w2v_model.build_vocab(sentences)
# Obtener las palabras del vocabulario
words = list(w2v_model.wv.key_to_index.keys())
print (len(words))

4318


In [12]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
w2v_model.init_sims(replace=True)

<ipython-input-12-ad53caad7e39>:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [13]:
w2v_model.wv.most_similar(positive=["homer"])

[('marge', 0.4529670774936676),
 ('dad', 0.43577075004577637),
 ('homie', 0.42787420749664307),
 ('dear_lord', 0.40275895595550537),
 ('reverend_lovejoy', 0.38601261377334595),
 ('moe', 0.37758636474609375),
 ('herb', 0.3596359193325043),
 ('shhh', 0.3595944046974182),
 ('becky', 0.35933151841163635),
 ('mr_burns', 0.34440284967422485)]

In [22]:
w2v_model.wv.most_similar(negative=["homer"])

[('pole', 0.2755662798881531),
 ('followed', 0.2625522017478943),
 ('uses', 0.2499224841594696),
 ('played', 0.2443952113389969),
 ('leaves', 0.23739273846149445),
 ('texas', 0.23497380316257477),
 ('congress', 0.22601699829101562),
 ('globe', 0.22587725520133972),
 ('florida', 0.2249550223350525),
 ('pageant', 0.22243168950080872)]

In [15]:
w2v_model.wv.most_similar(positive=["homer", "marge", "bart", "lisa", "bart"])

[('mom', 0.5729303956031799),
 ('homie', 0.5428183674812317),
 ('maggie', 0.5224640369415283),
 ('dad', 0.5154610276222229),
 ('milhouse', 0.4808061122894287),
 ('honey', 0.4789040982723236),
 ('jessica', 0.47584906220436096),
 ('dr_hibbert', 0.46337488293647766),
 ('becky', 0.45037299394607544),
 ('your_father', 0.4401063323020935)]

In [28]:
import plotly.graph_objs as go
from sklearn.decomposition import PCA
import numpy as np

# Seleccionar palabra específica y sus vecinas (aumentamos el número de palabras vecinas)
target_word = 'homer'  # Reemplaza con la palabra de interés
neighbors = w2v_model.wv.most_similar(target_word, topn=50)  # Obtener las 50 palabras vecinas más similares
words = [target_word] + [neighbor[0] for neighbor in neighbors]
word_vectors = np.array([w2v_model.wv[word] for word in words])

# Reducir dimensiones a 3D con PCA
pca = PCA(n_components=3)
word_vectors_pca = pca.fit_transform(word_vectors)

# Crear trazado 3D con Plotly
trace = go.Scatter3d(
    x=word_vectors_pca[:, 0],
    y=word_vectors_pca[:, 1],
    z=word_vectors_pca[:, 2],
    mode='markers+text',
    text=words,
    marker=dict(size=5, color='blue', opacity=0.8),  # Ajustar el tamaño y la opacidad
    textposition="top center"
)

# Destacar la palabra objetivo en rojo
trace_target = go.Scatter3d(
    x=[word_vectors_pca[0, 0]],
    y=[word_vectors_pca[0, 1]],
    z=[word_vectors_pca[0, 2]],
    mode='markers+text',
    text=[target_word],
    marker=dict(size=10, color='red'),
    textposition="top center"
)

# Crear el layout
layout = go.Layout(
    title="Visualización de un Espacio Vectorial Más Grande",
    scene=dict(
        xaxis_title="PCA 1",
        yaxis_title="PCA 2",
        zaxis_title="PCA 3"
    ),
    margin=dict(l=0, r=0, b=0, t=50),
)

# Combinar los trazos
fig = go.Figure(data=[trace, trace_target], layout=layout)

# Mostrar la gráfica interactiva
fig.show()


In [24]:
import plotly.graph_objs as go
import numpy as np
from scipy.spatial.distance import cosine, cityblock

# Seleccionar las palabras de interés
words_of_interest = ['homer', 'marge', 'bart', 'lisa']  # Puedes agregar más palabras

# Obtener los vectores de las palabras
word_vectors = np.array([w2v_model.wv[word] for word in words_of_interest])

# Crear trazado 3D con Plotly usando las dimensiones originales
trace_words = go.Scatter3d(
    x=word_vectors[:, 0],
    y=word_vectors[:, 1],
    z=word_vectors[:, 2],  # Usamos las primeras 3 dimensiones
    mode='markers+text',
    text=words_of_interest,
    marker=dict(size=10, color='blue'),
    textposition="top center"
)

# Lista para almacenar las líneas entre palabras
trace_lines = []

# Calcular y dibujar líneas para cada par de palabras
for i in range(len(words_of_interest)):
    for j in range(i+1, len(words_of_interest)):
        # Obtener los vectores de las palabras
        vector1 = word_vectors[i]
        vector2 = word_vectors[j]

        # Calcular distancias
        euclidean_distance = np.linalg.norm(vector1 - vector2)
        cosine_similarity = 1 - cosine(vector1, vector2)  # 1 - cosine para que 1 sea más similar
        manhattan_distance = cityblock(vector1, vector2)

        # Mostrar las distancias en la consola
        print(f"Distancias entre '{words_of_interest[i]}' y '{words_of_interest[j]}':")
        print(f"  Euclidiana: {euclidean_distance}")
        print(f"  Similitud de coseno: {cosine_similarity}")
        print(f"  Distancia Manhattan: {manhattan_distance}")
        print()

        # Dibujar línea entre las palabras para representar la distancia euclidiana
        trace_line = go.Scatter3d(
            x=[vector1[0], vector2[0]],
            y=[vector1[1], vector2[1]],
            z=[vector1[2], vector2[2]],
            mode='lines',
            line=dict(color='gray', width=2),
            text=f"Euclidiana: {euclidean_distance:.2f}",
            hoverinfo="text"
        )
        trace_lines.append(trace_line)

# Crear el layout
layout = go.Layout(
    title="Comparación de Distancias en el Espacio Vectorial",
    scene=dict(
        xaxis_title="Vector Dimension 1",
        yaxis_title="Vector Dimension 2",
        zaxis_title="Vector Dimension 3"
    ),
    margin=dict(l=0, r=0, b=0, t=50),
)

# Combinar los trazos de puntos y líneas
fig = go.Figure(data=[trace_words] + trace_lines, layout=layout)

# Mostrar la gráfica interactiva
fig.show()


Distancias entre 'homer' y 'marge':
  Euclidiana: 1.045975923538208
  Similitud de coseno: 0.4529670802970489
  Distancia Manhattan: 14.347789764404297

Distancias entre 'homer' y 'bart':
  Euclidiana: 1.157731056213379
  Similitud de coseno: 0.3298293853009504
  Distancia Manhattan: 16.304750442504883

Distancias entre 'homer' y 'lisa':
  Euclidiana: 1.1677700281143188
  Similitud de coseno: 0.3181565918735775
  Distancia Manhattan: 15.684782981872559

Distancias entre 'marge' y 'bart':
  Euclidiana: 1.1976268291473389
  Similitud de coseno: 0.28284506303995527
  Distancia Manhattan: 16.26519012451172

Distancias entre 'marge' y 'lisa':
  Euclidiana: 1.1392414569854736
  Similitud de coseno: 0.35106442266318727
  Distancia Manhattan: 15.848823547363281

Distancias entre 'bart' y 'lisa':
  Euclidiana: 0.8958233594894409
  Similitud de coseno: 0.598750221878268
  Distancia Manhattan: 12.250713348388672



In [25]:
analogous_words = w2v_model.wv.most_similar(positive=['bart', 'marge'], negative=['homer'], topn=1)
print("La palabra que completa la analogía 'homero es a bart como marge es a ...' es:", analogous_words[0][0])

La palabra que completa la analogía 'homero es a bart como marge es a ...' es: lisa


In [39]:
palabras = ['marge', 'lisa', 'bart', 'homer', 'maggie', 'orange']
palabra_fuera = w2v_model.wv.doesnt_match(palabras)
print(f"La palabra que no encaja en el grupo {palabras} es: {palabra_fuera}")

La palabra que no encaja en el grupo ['marge', 'lisa', 'bart', 'homer', 'maggie', 'orange'] es: orange
